In [ ]:
!pip install -U datasets
!pip install huggingface-hub

In [ ]:
!huggingface-cli login

In [3]:
from huggingface_hub import HfApi, Repository, HfFolder
from datasets import concatenate_datasets, load_dataset

In [4]:
splits = ['train', 'validation', 'test', 'other', 'validated']
datasets = {split: load_dataset("mozilla-foundation/common_voice_17_0", "en", split=split, streaming=True) for split in splits}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for mozilla-foundation/common_voice_17_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_17_0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [13]:
ds = load_dataset("mozilla-foundation/common_voice_17_0", "en", split='train[:]',streaming=False)

OSError: [Errno 28] No space left on device

In [5]:
datasets

{'train': IterableDataset({
     features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
     n_shards: 28
 }),
 'validation': IterableDataset({
     features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
     n_shards: 1
 }),
 'test': IterableDataset({
     features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
     n_shards: 1
 }),
 'other': IterableDataset({
     features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
     n_shards: 9
 }),
 'validated': IterableDataset({
     features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment', 'variant'],
     n_shards: 45
 })}

In [5]:
cols=[col for col in datasets["train"].column_names if (col != "audio" and col !="sentence")]

In [6]:
for split in splits:
  datasets[split] = datasets[split].remove_columns(cols)

In [7]:
for split in splits:
  datasets[split] = datasets[split].rename_column("sentence", "text")

In [11]:
datasets

{'train': IterableDataset({
     features: ['audio', 'text'],
     n_shards: 28
 }),
 'validation': IterableDataset({
     features: ['audio', 'text'],
     n_shards: 1
 }),
 'test': IterableDataset({
     features: ['audio', 'text'],
     n_shards: 1
 }),
 'other': IterableDataset({
     features: ['audio', 'text'],
     n_shards: 9
 }),
 'validated': IterableDataset({
     features: ['audio', 'text'],
     n_shards: 45
 })}

In [8]:
splitsection= [datasets[split] for split in splits]

In [19]:
splitsection

[IterableDataset({
     features: ['audio', 'text'],
     n_shards: 28
 }),
 IterableDataset({
     features: ['audio', 'text'],
     n_shards: 1
 }),
 IterableDataset({
     features: ['audio', 'text'],
     n_shards: 1
 }),
 IterableDataset({
     features: ['audio', 'text'],
     n_shards: 9
 }),
 IterableDataset({
     features: ['audio', 'text'],
     n_shards: 45
 })]

In [9]:
token = HfFolder.get_token()

In [10]:
api = HfApi()

In [11]:
repo_id = ""

In [ ]:
api.create_repo(repo_id, exist_ok=True)

In [ ]:
from functools import partial
from datasets import Dataset

def gen_from_iterable_dataset(iterable_ds):
    yield from iterable_ds

ds = Dataset.from_generator(partial(gen_from_iterable_dataset, splitsection[0]), features=splitsection[0].features)

In [ ]:
ds.push_to_hub(repo_id)